# 한국 맛집 어플 데이터를 크롤링 후 학습

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import warnings
import os


rating_file_path = os.getenv('HOME') +'/aiffel/H3/foodrating/app_db.csv'
#ratings_cols = ['ProductId', 'ProfileName', 'Score', 'Summary']
data1 = pd.read_csv(rating_file_path, header = None, index_col = None)
data1.columns = ['userId','Menu','Rating']
data1.head()
#data1.tail()

,userId,Menu,Rating
0,수염시,알밥,5
1,수염시,회,5
2,수염시,치킨,5
3,수염시,닭발,5
4,수염시,물냉면,5


In [2]:
data1["userId"].nunique()

40

# 크게 두가지 모델로 나눌수 있습니다 : 
# 1) 아이템 기반 추천시스템 2) 유저 기반 추천시스템

# 유저기반 추천시스템

In [3]:
user_df = pd.read_csv(rating_file_path, header = None, index_col = None)
user_df.columns = ['userId','Menu','Rating']
user_df.head()

,userId,Menu,Rating
0,수염시,알밥,5
1,수염시,회,5
2,수염시,치킨,5
3,수염시,닭발,5
4,수염시,물냉면,5


In [4]:
user_df["userId"].nunique()

40

# 인기 많은 음식들 (top 30)

In [5]:
product_count = user_df.groupby('Menu')['userId'].count()
product_count.sort_values(ascending = False).head(30)

Menu
치킨       28
떡볶이      20
피자       20
족발       17
삼겹살      14
탕수육      13
짬뽕       12
닭발       11
회        10
돈까스      10
김치찌개      8
찜닭        8
닭도리탕      8
햄버거       7
보쌈        7
마라탕       7
야채곱창      6
샌드위치      5
라멘        5
파스타       5
초밥        4
육회        4
짜장면       4
김밥        4
돼지김치찜     4
물냉면       4
막창구이      4
닭강정       3
와플        3
곱창구이      3
Name: userId, dtype: int64

# 3. 내가 선호하는 식품 7가지 골라서 rating 에 추가

In [6]:

my_favorite = ['치킨', '햄버거', '족발','회', '돼지김치찜', '삼겹살', '돈까스']

my_foodlist = pd.DataFrame({'userId': ['규환']*7,  'Rating':[5]*7,
                           'Menu': ['치킨', '햄버거', '족발','회', '돼지김치찜', '삼겹살', '돈까스']})

if not user_df.isin({'userId': ['규환']})['userId'].any():
    user_df = user_df.append(my_foodlist)
user_df.tail(10)

,userId,Menu,Rating
354,손님,막창구이,5
355,손님,닭발,5
356,손님,피자,5
0,규환,치킨,5
1,규환,햄버거,5
2,규환,족발,5
3,규환,회,5
4,규환,돼지김치찜,5
5,규환,삼겹살,5
6,규환,돈까스,5


In [7]:
# 고유한 유저, 프로덕트 찾기
user_unique = user_df['userId'].unique()
product_unique = user_df['Menu'].unique()

# 유저, 프로덕트를 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
product_to_idx = {v:k for k,v in enumerate(product_unique)}

print(user_to_idx['규환'])

40


In [8]:
# CSR Matrix 를 직접 만들어 보기


# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# user_to_idx.get을 통해 UserId 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 

temp_user_data = user_df['userId'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(user_df):   # 모든 row가 정상적으로 인덱싱되었다면
    print('유저 ID 컬럼 인덱싱 완료!!')
    user_df['userId'] = temp_user_data   # data['UserId']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('유저 ID 컬럼 인덱싱 실패')

# product_to_idx을 통해 product 컬럼도 동일한 방식으로 인덱싱해 줍니다.

temp_product_data = user_df['Menu'].map(product_to_idx.get).dropna()
if len(temp_product_data) == len(user_df):
    print('메뉴 컬럼 인덱싱 완료!!')
    user_df['Menu'] = temp_product_data
else:
    print('메뉴 컬럼 인덱싱 실패!!')
    

user_df

유저 ID 컬럼 인덱싱 완료!!
메뉴 컬럼 인덱싱 완료!!


,userId,Menu,Rating
0,0,0,5
1,0,1,5
2,0,2,5
3,0,3,5
4,0,4,5
...,...,...,...
2,40,5,5
3,40,1,5
4,40,19,5
5,40,10,5


# Compressed Sparse Row Matrix

유저 x 아이템 평가행렬 -> 어림잡아도 엄청난 메모리가 필요하다 (600GB 이상)

유저가 좋아하지 않는 아이템에 대한 정보까지 모두 행렬에 포함되어 계산되기 때문.

평가행렬 내의 대부분의 공간은 0으로 채워짐. 이를 Sparse Matrix 라고 부름.

이런 메모리 낭비를 최소화 하기 위해 유저가 좋아하는 아이템에 대해서만 정보만을 저장하면서

전체 행렬 형태를 유추할 수있는 데이터 구조가 필요

# Sparse Matrix 에 관한 URL :

1) https://stackoverflow.com/questions/53254104/cant-understand-scipy-sparse-csr-matrix-example/62118005#62118005

2)https://lovit.github.io/nlp/machine%20learning/2018/04/09/sparse_mtarix_handling/#csr-matrix

In [9]:
#CSR MATRIX 만들기
from scipy.sparse import csr_matrix

num_user = user_df['userId'].nunique()
num_product = user_df['Menu'].nunique()

csr_data = csr_matrix((user_df['Rating'], (user_df.userId, user_df.Menu)), shape= (num_user, num_product))
csr_data

<41x107 sparse matrix of type '<class 'numpy.longlong'>'
	with 364 stored elements in Compressed Sparse Row format>

# Matrix Factorization 모델 학습

implict 라는 패키지를 이용

als(AlternatingLeastSqaures) 모델을 사용. Matrix Factorization 에서 쪼개진 두 Feature Matrix 를 한꺼번에 훈련하는것은 잘 수렴하지 않기 때문에,
한쪽을 고정시키고 다른 쪽을 학습하는 방식을 번갈아 수행하는 ALS 방식이 효과적임

In [10]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

#implict 에서 권장하고 있는 부분
os.environ['OPENBLAS_NUM_THREAD'] = '1'
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
os.environ['MKL_NUM_THREADS'] = '1'

#implict AlternatingLeastSquares 모델의 선언


#ALS 클래스의 __init__ 파라미터 살펴보기
# 1. factors : 유저와 아이템의 벡터를 몇 차원으로 할것인지
# 2. regularization : 과적합 방지하기 위해 정규화 값을 얼마나 사용할 것인지
# 3. use_gpu : GPU 를 사용할 것잉ㄴ지
# 4. iterations : epoch 과 같은 의미. 데이터를 몇 번 반복해서 학습할 것인지
als_model = AlternatingLeastSquares(factors = 100, regularization = 0.01, use_gpu = False, iterations = 15,
                                   dtype = np.float32)

#als 모델은 input 으로 (item X user 꼴의 matrix 를 받기 때문에 Transpose 해줍니다.)

csr_data_transpose = csr_data.T
csr_data_transpose

<107x41 sparse matrix of type '<class 'numpy.longlong'>'
	with 364 stored elements in Compressed Sparse Column format>

In [11]:
als_model.fit(csr_data_transpose)

In [12]:
print(user_to_idx['규환'])

40


# 4.  내가 선호하는 7가지 음식 중 하나와 그 외의 음식 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해보기

In [13]:
규환, 족발 = user_to_idx['규환'], product_to_idx['족발']
규환_vector, 족발_vector = als_model.user_factors[규환], als_model.item_factors[족발]

In [14]:
규환_vector

array([ 0.69995874,  0.738884  , -1.3741947 , -0.26483878,  2.07991   ,
        0.7183035 ,  0.54025245,  0.94908947,  1.049285  ,  0.34145996,
        0.93235445,  0.7154961 ,  1.0814351 ,  1.0539453 ,  1.5912566 ,
        0.41928175,  0.980873  ,  1.069271  ,  2.081347  ,  0.5199263 ,
        0.24170259,  0.24863797, -0.48982242, -0.51139295,  1.4662756 ,
        0.25783893,  1.2389318 , -1.5528257 ,  0.5626008 ,  0.20349845,
       -0.05996896,  1.138958  , -0.5395856 ,  0.5764024 ,  0.3445158 ,
        1.0366554 , -0.22900985,  0.36845917,  1.6440117 ,  2.1554594 ,
        0.26593056,  1.0556773 , -0.33711204, -0.74362123,  0.1796091 ,
       -0.9657275 ,  1.4584912 ,  1.7142336 ,  0.08038958,  0.34740552,
        0.30779564, -0.37768018,  0.1076901 , -0.40854853, -0.98308337,
       -0.24585418,  0.93128973, -0.28908008, -1.1502423 ,  0.6840869 ,
        1.1249542 ,  1.0177773 , -0.56783015,  0.9124722 ,  0.6230983 ,
        0.13071334,  1.4557842 ,  0.04785869,  0.48292047,  0.02

In [15]:
족발_vector

array([ 0.02280351,  0.04081863, -0.01353618, -0.03109088, -0.01255145,
        0.00488367,  0.03052302,  0.03265673,  0.0508211 , -0.01532242,
        0.01295061,  0.02676212, -0.00191345,  0.02118581, -0.00641235,
        0.00103157, -0.00733556, -0.00299384,  0.00465611,  0.00925324,
       -0.01592219, -0.04736574, -0.02042919, -0.01820992, -0.01558553,
       -0.02609957,  0.04779591, -0.04637858, -0.00804448,  0.00783718,
       -0.01639334,  0.05905644, -0.01942983,  0.01203618, -0.01072067,
        0.02205144,  0.02243058, -0.00407049,  0.04632495,  0.0434833 ,
       -0.00592416,  0.04069965, -0.01781283, -0.00950384,  0.02230106,
        0.00933353,  0.00033016,  0.00809464,  0.04744013, -0.02617015,
        0.00544009,  0.05123416,  0.00423148, -0.01850438, -0.01274553,
        0.01751447, -0.00507137, -0.01483055,  0.00887226,  0.01020309,
        0.01782646, -0.04401979, -0.01635353,  0.0506948 ,  0.0068062 ,
        0.04107188,  0.05613685, -0.00806735, -0.0075924 ,  0.00

In [16]:
# 규환과 족발이 내적하는 코드

np.dot(규환_vector, 족발_vector)

1.0020393

In [17]:
# 나의 회에 대한 선호도는 어떻게 예측할지

회 = product_to_idx['회']
회_vector = als_model.item_factors[회]
np.dot(규환_vector, 회_vector)

0.9949883

# 5. 내가 좋아하는 음식과 비슷한 음식을 추천받아 봅시다.

In [18]:
# 삼결살과 비슷한 음식 추천받기

favorite_food = '삼겹살'
product_id = product_to_idx[favorite_food]

similar_food = als_model.similar_items(product_id, N=15)
similar_food

[(10, 1.0),
 (17, 0.18438953),
 (20, 0.18282984),
 (21, 0.18035649),
 (22, 0.17982247),
 (85, 0.1738568),
 (86, 0.17028402),
 (53, 0.14748503),
 (61, 0.14264074),
 (59, 0.14205591),
 (80, 0.13368067),
 (11, 0.1302035),
 (28, 0.122482546),
 (74, 0.11932728),
 (12, 0.11775661)]

In [19]:
#음식 이름으로 변경해주기

idx_to_product = {v:k for k,v in product_to_idx.items()}
[idx_to_product[i[0]]for i in similar_food]

['삼겹살',
 '짬뽕',
 '짜글이',
 '오징어튀김',
 '불고기',
 '술국',
 '리조또',
 '갈비찜',
 '곰탕',
 '샤브샤브',
 '와플',
 '파스타',
 '돈까스',
 '팥빙수',
 '콩나물불고기']

# 6. 내가 좋아할만한 음식들을 추천받아보기

In [20]:
# 음식 추천받기

user = user_to_idx['규환']

#recommend 에서는 user * item CSR Matrix 를 받습니다.

food_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items = True)
food_recommended

[(29, 0.021629795),
 (35, 0.017073952),
 (36, 0.016226284),
 (6, 0.01520206),
 (15, 0.01329278),
 (4, 0.012837097),
 (84, 0.011978179),
 (31, 0.01139693),
 (37, 0.0113159865),
 (95, 0.010678738),
 (60, 0.007741615),
 (59, 0.006372839),
 (24, 0.0063656867),
 (61, 0.0061508715),
 (83, 0.005716637),
 (74, 0.0055408664),
 (82, 0.00501699),
 (12, 0.0050051827),
 (27, 0.0047495477),
 (45, 0.0047326162)]

In [21]:
[idx_to_product[i[0]]for i in food_recommended]

['보쌈',
 '라멘',
 '쭈삼볶음',
 '피자',
 '곱창구이',
 '물냉면',
 '순대국',
 '초밥',
 '마라상궈',
 '비빕밥',
 '쌈밥',
 '샤브샤브',
 '김밥',
 '곰탕',
 '고로케',
 '팥빙수',
 '소라무침',
 '콩나물불고기',
 '육회',
 '감자탕']

In [22]:
#이 추천에 기여한 정도 (돈까스의 경우)

돈까스 = product_to_idx['돈까스']
explain = als_model.explain(user, csr_data, itemid = 돈까스)

[(idx_to_product[i[0]], i[1]) for i in explain [1]]

[('돈까스', 0.9120330259276284),
 ('돼지김치찜', 0.039507316587005153),
 ('삼겹살', 0.02285229967886593),
 ('햄버거', 0.015973619658981038),
 ('회', 0.008634504746996317),
 ('족발', -0.00954454685730343),
 ('치킨', -0.012980513261401753)]